In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
import cv2
import matplotlib.pyplot as plt
import random
import multiprocessing
from keras.backend import set_session
from keras.backend import clear_session
from keras.backend import get_session


In [2]:
# Disable gpu to use cpu
tf.config.set_visible_devices([], 'GPU')

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    # tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
    # config = tf.compat.v1.ConfigProto()
    # config.gpu_options.per_process_gpu_memory_fraction = 0.6
    # keras.set_session(tf.Session(config=config))
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [2]:
# Setup working directory
emotionList = ('Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt')
# limitCounter = [0,0,0,0,0,0,0,0]


def Setup():
  os.chdir("C:\\Users\\j.teoh\\Desktop\\tflite-facial-expression") # change working directory

# currently limited to 19999 images
# Load the images into a nested list containing info of image in tuple format (label, image name)
def LoadAllImageNames(imageFiles):
  trainingSet = []
  loadCounter = 0
  happySetCounter = 0
  for file in imageFiles:
    name = file.name[:-4] # file name w/o file extension
    data = np.load("train_set\\annotations\\{}_exp.npy".format(name))
    # if limitCounter[int(data.item(0))] > 3000:
    #   continue
    loadCounter += 1
    if (loadCounter%10000==0):
      print("Files loaded:{}".format(loadCounter))
    # limitCounter[int(data.item(0))] += 1
    label = int(data.item(0))
    if label != 1 and label != 2 and label != 6:
        continue
    if happySetCounter > 60000:
        continue
    happySetCounter += 1
    trainingSet.append((label, file.name))
  random.shuffle(trainingSet)
  return trainingSet

# Load the images into a nested list containing info of image in tuple format (label, image name)
def LoadAllTestImageNames(imageFiles):
  testingSet = []
  loadCounter = 0
  for file in imageFiles:
    loadCounter += 1
    if (loadCounter%10000==0):
      print("Files loaded:{}".format(loadCounter))
      if (loadCounter > 10000):
        break
    name = file.name[:-4] # file name w/o file extension
    data = np.load("val_set\\annotations\\{}_exp.npy".format(name))
    label = int(data.item(0))
    if label != 1 and label != 2 and label != 6:
        continue
    testingSet.append((label, file.name))

  random.shuffle(testingSet)

  return testingSet

# Load the pixels of a picture to numpy.ndarray format. false for test set, true for training set
# Return image in RGB format
def LoadImage(imageName, trainingSetBool, normalize = True):
  intermediatePath = str()
  if (trainingSetBool):
    intermediatePath = "\\train_set\\images\\"
  else:
    intermediatePath = "\\val_set\\images\\"
  if normalize:
    image_array = cv2.imread("{}{}{}".format(os.getcwd(), intermediatePath, imageName))
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
    image_array = image_array/255
  else:
    image_array = cv2.imread("{}{}{}".format(os.getcwd(), intermediatePath, imageName))
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
  return image_array

# Extract the daata from 0 to amount from list and return it
def CropData(list, amount):
  if (len(list) < amount):
    amount = len(list)
  croppedList = list[:amount]
  del list[:amount]
  return croppedList

def LoadImages(list, trainingSetBool):
  label = []
  data = []
  count = 0
  for entries in list:
    try:
      image = LoadImage(entries[1], trainingSetBool)
      data.append(image)
      label.append(entries[0])
    except:
      print("Failed to load training image: ", entries[1])
  npLabel = np.array(label)
  npData = np.array(data)
  return npLabel, npData

def InitializeModel():
  pretrained_model = tf.keras.applications.MobileNetV3Large(input_shape=(224,224,3)) # Initializing model with mobile net V3 pretrained model

  # Initializing the input and output from the model, removing last layer
  base_input = pretrained_model.layers[0].input
  base_output = pretrained_model.layers[-2].output

  # Adding 3 more layers to output side
  final_output = layers.Dense(128)(base_output) # Adding new layers, to the output side
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(64)(final_output)
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(8, activation = 'softmax')(final_output) # 8 cuz there are 8 image classifications

  new_model = keras.Model(inputs = base_input, outputs = final_output)
  return new_model

def ConvertToGray(image):
  image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
  return image

def ScaleImage(image, width):
  ratio = image.shape[1]/width
  image = cv2.resize(image, (width, int(image.shape[0]/ratio)))
  return image

def DetectFace(image):
  face_roi = np.ndarray(1)
  faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
  grayImage = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  faces = faceCascade.detectMultiScale(grayImage, 1.3, 5)
  for x,y,w,h in faces:
    roi_gray = grayImage[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]
    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 2)
    facess = faceCascade.detectMultiScale(roi_gray)
    if (len(facess) == 0):
      print("Face not detected")
    else:
      for (ex,ey,ew,eh) in facess:
        face_roi = roi_color[ey:ey+eh, ex:ex+ew]
  return face_roi

def ConvertToInput(image):
  input = ScaleImage(image, 224)
  input = np.expand_dims(input, axis = 0) ## to add fourth dimension to fit model input
  input = input/255
  return input

def GetResult(model, input):
  Predictions = model.predict(input)
  print(Predictions)
  result = np.argmax(Predictions)
  return emotionList[result]

def trainModel(dataList):
  print("ran")

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    # try:
    #     del classifier # this is from global space - change this as you need
    # except:
    #     pass

    # use the same config as you used to create the session
    tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9, visible_device_list="0")

In [10]:
# Get currect directory (os.getcwd() -> C:\Users\jazzt\src)

#-----------------------Start of code---------------------------
# Path directories
Setup()

# initialise image names and label
imageFiles = os.scandir("train_set\\images")
trainingSetData = LoadAllImageNames(imageFiles)

Files loaded:10000
Files loaded:20000
Files loaded:30000
Files loaded:40000
Files loaded:50000
Files loaded:60000
Files loaded:70000
Files loaded:80000
Files loaded:90000
Files loaded:100000
Files loaded:110000
Files loaded:120000
Files loaded:130000
Files loaded:140000
Files loaded:150000
Files loaded:160000
Files loaded:170000
Files loaded:180000
Files loaded:190000
Files loaded:200000
Files loaded:210000
Files loaded:220000
Files loaded:230000
Files loaded:240000
Files loaded:250000
Files loaded:260000
Files loaded:270000
Files loaded:280000


In [11]:
# Default loading in model and training with reset of keras in GPU every iteration
# initialise model
model = InitializeModel()
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

# train model

batchSize = 48
imgPerIter = batchSize*16
count = 0
while(len(trainingSetData) != 0):
  # training data
  try:
    croppedList = CropData(trainingSetData, imgPerIter)
    print("loading image")
    label, data = LoadImages(croppedList, True)
    model.fit(data, label, epochs = 11, batch_size = batchSize)
  except RuntimeError as e:
    print(e)
  
  model.save_weights('label126Limited6BatchSize48gpu.h5')
  count = count + imgPerIter
  print("trained image count: ", count)
  reset_keras()


loading image
Epoch 1/11
16/16 [==============================] - 50s 182ms/step - loss: 0.8608 - accuracy: 0.6797
Epoch 2/11
16/16 [==============================] - 3s 183ms/step - loss: 0.3635 - accuracy: 0.8672
Epoch 3/11
16/16 [==============================] - 3s 182ms/step - loss: 0.1374 - accuracy: 0.9583
Epoch 4/11
16/16 [==============================] - 3s 183ms/step - loss: 0.0602 - accuracy: 0.9818
Epoch 5/11
16/16 [==============================] - 3s 183ms/step - loss: 0.1016 - accuracy: 0.9740
Epoch 6/11
16/16 [==============================] - 3s 184ms/step - loss: 0.1440 - accuracy: 0.9544
Epoch 7/11
16/16 [==============================] - 3s 187ms/step - loss: 0.1772 - accuracy: 0.9349
Epoch 8/11
16/16 [==============================] - 3s 184ms/step - loss: 0.1117 - accuracy: 0.9727
Epoch 9/11
16/16 [==============================] - 3s 184ms/step - loss: 0.0409 - accuracy: 0.9870
Epoch 10/11
16/16 [==============================] - 3s 183ms/step - loss: 0.0540 - a

In [ ]:
# load and save weights every iter, reset gpu memory with cuda (will break the program)


# train model

count = 0
while(len(trainingSetData) != 0):
  model = InitializeModel()
  model.load_weights("feModelWeights2feb.h5")
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  # training data
  try:
    croppedList = CropData(trainingSetData, 1000)
    print("loading image")
    label, data = LoadImages(croppedList, True)
    model.fit(data, label, epochs = 11, batch_size = 32)
  except RuntimeError as e:
    print(e)
  
  model.save_weights('feModelWeights2feb.h5')
  count = count + 1
  print("trained image count: ", count*1000)
  cuda.get_current_device().reset()


In [ ]:
#using multiprocessor to run model
# train model
count = 0
while(len(trainingSetData) != 0):

    croppedList = CropData(trainingSetData, 1000)
    print("loading image")
    p = multiprocessing.Process(target = trainModel, args=(croppedList))
    p.start()
    p.join()
    
  
    count = count + 1
    print("trained image count: ", count*1000)


In [ ]:
# Default loading and training of model, extracting lower half of data only

#-----------------------Start of code---------------------------
# Path directories
Setup()

# initialise image names and label
imageFiles = os.scandir("train_set\\images")
trainingSetData = LoadAllImageNames(imageFiles)

# initialise model
model = InitializeModel()
model.load_weights("feModelWeights5feb.h5")
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
trainingSetData = trainingSetData[(120000+161000):]

# train model
count = 0
while(len(trainingSetData) != 0):
  # training data
  try:
    croppedList = CropData(trainingSetData, 1000)
    print("loading image")
    label, data = LoadImages(croppedList, True)
    model.fit(data, label, epochs = 11, batch_size = 32)
  except RuntimeError as e:
    print(e)
  
  model.save_weights('feModelWeights6feb.h5')
  count = count + 1
  print("trained image count: ", count*1000)


In [ ]:
# Count number of data in each facial expression
imageFiles = os.scandir("train_set\\images")
trainingSetData = LoadAllImageNames(imageFiles)

counterList = list(range(8))
for label, name in trainingSetData:
    counterList[label] += 1
    
print(counterList)
    

In [ ]:
sum = 0
for i in counterList:
    sum += i

for i in range(len(counterList)):
    counterList[i] /= sum

print(counterList)

In [ ]:
croppedList = CropData(trainingSetData, 1000)
label, data = LoadImages(croppedList, True)

In [ ]:
model = InitializeModel()
print(model.summary())
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

In [ ]:
model = InitializeModel()
print(model.summary())
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

In [ ]:
model.fit(data, label, epochs = 13, batch_size = 4)

In [ ]:
# Testing trained model
image = LoadImage("3.jpg", False, normalize = False)
# image = cv2.resize(image, (224,224))
image = ScaleImage(image, 224)

face = DetectFace(image)

plt.imshow(face)

preppedInput = ConvertToInput(face)
result = GetResult(model, preppedInput)


In [ ]:
result

In [ ]:
# testing loading in data from tar using tensorflow
os.chdir("C:\\Users\\j.teoh\\Desktop\\tflite-facial-expression") # change working directory
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)

In [ ]:
images, labels = next(img_gen.flow_from_directory("train_set.tar"))

In [ ]:
croppedList = CropData(trainingSetData, 1000)
label, data = LoadImages(croppedList, True)
dataset = tf.data.Dataset.from_tensor_slices((data,label)).batch(1)

In [ ]:
def show(image, label):
  plt.figure()
  plt.imshow(image)
  plt.title(label)
  plt.axis('off')

In [ ]:
model = InitializeModel()
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])


In [ ]:
model.fit(dataset.repeat(), epochs=11, steps_per_epoch=20)

In [3]:
# Testing model
Setup()

modelNames = ["label126Limited6BatchSize48gpu.h5"]
model = InitializeModel()
mainData = LoadAllTestImageNames(os.scandir("val_set\\images"))
resultPool =[]

for modelName in modelNames:
    model.load_weights(modelName)
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

    # initialise image names and label
    testSetData = []
    testSetData.extend(mainData)
    # print(mainData)
    # test model
    lostSum = 0
    accuracySum = 0
    count = 0
    while(len(testSetData) != 0):
      # training data
      # try:
      croppedList = CropData(testSetData, 100)
      print("loading image")
      label, data = LoadImages(croppedList, False)
      result = model.evaluate(data, label, batch_size = 1)
      lostSum += result[0]
      accuracySum += result[1]
      # except:
      #   print("Failed to train data")

      count += 1
      reset_keras()
      # print(count)
    # print(mainData)
    print("==========FINISH TESTING===========")
    print("model name: ", modelName)
    print("average lost: ", lostSum/count)
    print("average accuracy: ", accuracySum/count)
    resultPool.append((modelName, lostSum/count, accuracySum/count))
    
print(resultPool)

loading image
100/100 [==============================] - 14s 90ms/step - loss: 1.1848 - accuracy: 0.3100
loading image
100/100 [==============================] - 9s 89ms/step - loss: 1.1544 - accuracy: 0.4300
loading image
100/100 [==============================] - 9s 90ms/step - loss: 1.0759 - accuracy: 0.4200
loading image
100/100 [==============================] - 9s 90ms/step - loss: 1.1282 - accuracy: 0.4200
loading image
100/100 [==============================] - 9s 91ms/step - loss: 1.0483 - accuracy: 0.4400
loading image
100/100 [==============================] - 9s 90ms/step - loss: 1.0494 - accuracy: 0.4400
loading image
100/100 [==============================] - 9s 90ms/step - loss: 1.0428 - accuracy: 0.5200
loading image
100/100 [==============================] - 9s 88ms/step - loss: 1.0847 - accuracy: 0.4300
loading image
100/100 [==============================] - 9s 90ms/step - loss: 1.1960 - accuracy: 0.3800
loading image
100/100 [==============================] - 9s 89m